In [1]:
import requests
import base64
import os
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime

# 載入環境變數
load_dotenv()

# =============================================================================
# 步驟 1: 用 Refresh Token 獲取 Access Token
# =============================================================================

def get_access_token():
    """用 refresh_token 獲取新的 access_token"""
    
    CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
    CLIENT_SECRET = os.getenv('SPOTIFY_CLIENT_SECRET')
    REFRESH_TOKEN = os.getenv('SPOTIFY_REFRESH_TOKEN')
    
    # 編碼 credentials
    auth_str = f"{CLIENT_ID}:{CLIENT_SECRET}"
    auth_base64 = base64.b64encode(auth_str.encode()).decode()
    
    # 請求新的 access token
    response = requests.post(
        "https://accounts.spotify.com/api/token",
        headers={
            'Authorization': f'Basic {auth_base64}',
            'Content-Type': 'application/x-www-form-urlencoded'
        },
        data={
            'grant_type': 'refresh_token',
            'refresh_token': REFRESH_TOKEN
        }
    )
    
    if response.status_code == 200:
        tokens = response.json()
        print(f"✅ Access Token 獲取成功 (有效期: {tokens['expires_in']} 秒)")
        return tokens['access_token']
    else:
        print(f"❌ 獲取失敗: {response.status_code}")
        print(response.text)
        return None

In [2]:
def get_recently_played(access_token, limit=20):
    """
    獲取最近播放的歌曲
    
    參數:
        access_token: 從 get_access_token() 獲得
        limit: 要獲取的歌曲數量 (最多 50)
    
    返回:
        DataFrame 包含播放記錄
    """
    
    print(f"\n🎵 正在獲取最近播放的 {limit} 首歌...")
    
    url = "https://api.spotify.com/v1/me/player/recently-played"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {"limit": limit}
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        items = data['items']
        
        print(f"✅ 成功獲取 {len(items)} 首歌曲\n")
        
        # 整理資料
        tracks_list = []
        
        for item in items:
            track = item['track']
            played_at = item['played_at']
            
            # 轉換時間格式
            played_time = datetime.fromisoformat(played_at.replace('Z', '+00:00'))
            
            tracks_list.append({
                'played_at': played_time,
                'track_id': track['id'],
                'track_name': track['name'],
                'artist': ', '.join([a['name'] for a in track['artists']]),
                'album': track['album']['name'],
                'duration_ms': track['duration_ms'],
                'popularity': track['popularity'],
                'spotify_url': track['external_urls']['spotify']
            })
        
        df = pd.DataFrame(tracks_list)
        return df
    
    elif response.status_code == 401:
        print("❌ Token 無效或過期")
        return None
    else:
        print(f"❌ 請求失敗: {response.status_code}")
        print(response.text)
        return None

In [3]:
access_token = get_access_token()
df = get_recently_played(access_token)

✅ Access Token 獲取成功 (有效期: 3600 秒)

🎵 正在獲取最近播放的 20 首歌...
✅ 成功獲取 20 首歌曲



In [4]:
df.head()

,played_at,track_id,track_name,artist,album,duration_ms,popularity,spotify_url
0,2025-10-13 14:52:08.044000+00:00,51Z4HhqC0DljkAmYeFDRmx,午後二時の通り雨,がらり,手のひら望遠鏡,186347,28,https://open.spotify.com/track/51Z4HhqC0DljkAm...
1,2025-10-13 14:49:02.099000+00:00,4h14WLXjufBxMC3gZwRmUY,青春写真,がらり,青春写真,237431,32,https://open.spotify.com/track/4h14WLXjufBxMC3...
2,2025-10-13 10:03:39.787000+00:00,4oE7MyJhqSD3BaHRpNs8Nl,JANE DOE,"Kenshi Yonezu, Hikaru Utada",JANE DOE,235946,82,https://open.spotify.com/track/4oE7MyJhqSD3BaH...
3,2025-10-13 09:57:09.584000+00:00,2fJQvVtZDlLZ3kS9Tdbw1x,逃避行,がらり,逃避行,213571,24,https://open.spotify.com/track/2fJQvVtZDlLZ3kS...
4,2025-10-12 23:50:07.811000+00:00,1SFErqsIANLo80NECd1gDF,ステラ,がらり,ステラ,213616,26,https://open.spotify.com/track/1SFErqsIANLo80N...


In [5]:
def get_track_info(track_id, access_token):
    url = f"https://api.spotify.com/v1/tracks/{track_id}"
    headers = {"Authorization": f"Bearer {access_token}"}
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ 獲取曲目資訊失敗: {response.status_code}")
        return None

In [8]:
re = get_track_info("51Z4HhqC0DljkAmYeFDRmx", access_token)

In [9]:
re

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0JLttnOJnnXSyy8xRFiCPp'},
    'href': 'https://api.spotify.com/v1/artists/0JLttnOJnnXSyy8xRFiCPp',
    'id': '0JLttnOJnnXSyy8xRFiCPp',
    'name': 'がらり',
    'type': 'artist',
    'uri': 'spotify:artist:0JLttnOJnnXSyy8xRFiCPp'}],
  'available_markets': ['AR',
   'AU',
   'AT',
   'BE',
   'BO',
   'BR',
   'BG',
   'CA',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DK',
   'DO',
   'DE',
   'EC',
   'EE',
   'SV',
   'FI',
   'FR',
   'GR',
   'GT',
   'HN',
   'HK',
   'HU',
   'IS',
   'IE',
   'IT',
   'LV',
   'LT',
   'LU',
   'MY',
   'MT',
   'MX',
   'NL',
   'NZ',
   'NI',
   'NO',
   'PA',
   'PY',
   'PE',
   'PH',
   'PL',
   'PT',
   'SG',
   'SK',
   'ES',
   'SE',
   'CH',
   'TW',
   'TR',
   'UY',
   'US',
   'GB',
   'AD',
   'LI',
   'MC',
   'ID',
   'JP',
   'TH',
   'VN',
   'RO',
   'IL',
   'ZA',
   'SA',
   'AE',
   'BH',
   'QA',
   'OM',
   'KW',